In [24]:
import torch
import numpy as np
import torch.nn as nn
import sys

sys.path.append('../src/')

import tforms
import utils
sys.path.append('../src/novelty_dfm_CL/')



import dset8_specific_scripts.classifier_8dset as clf

network = clf.Resnet_PSP([104, 72, 38], 3, resnet_arch='resnet50_contrastive', FC_layers=[7096,4098], base_freeze=True)

network = network.cuda()
   
def predict_task_baseline_peakedness(num_tasks, image, network, peakedness_m='max'):
    '''Predict a task assignment using peakedness measure'''
        
    image = image.cuda()
       
    softmax = nn.Softmax(dim=-1) ## find out the right dimension
                
    with torch.no_grad():
        values=[]
        for tt in range(num_tasks):
            task_embedding = network(image, tt, base_apply=False) ## generate task_num different embeddings 

            Y_hat = softmax(task_embedding)
            
            print('Y_hat', Y_hat.shape)
            if peakedness_m=='entropy':
                entropy = -1*torch.sum(Y_hat * torch.log(Y_hat))
                values.append(entropy)
            elif peakedness_m=='max':
                values.append(Y_hat.max())

        print('values', torch.stack(values))
        
        if peakedness_m=='entropy':
            task_predicted = torch.argmin(torch.stack(values))
        elif peakedness_m=='max':
            task_predicted = torch.argmax(torch.stack(values))
        
    return int(task_predicted.item())
    
    
    
    
    
    
    
    
class BinaryHashLinear(nn.Module):
    def __init__(self, n_in, n_out, period, key_pick='hash', learn_key=True):
        super(BinaryHashLinear, self).__init__()
        self.key_pick = key_pick
        w = nn.init.xavier_normal_(torch.empty(n_in, n_out))
        rand_01 = np.random.binomial(p=.5, n=1, size=(n_in, period)).astype(np.float32)
        o = torch.from_numpy(rand_01*2 - 1)

        self.w = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(n_out))
        self.o = nn.Parameter(o)
        if not learn_key:
            self.o.requires_grad = False

    def forward(self, x, task):
        o = self.o[:, task]
        m = x*o
        r = torch.mm(m, self.w)
        return r
    
    
class BinaryHashLinear(nn.Module):
    def __init__(self, n_in, n_out, period, key_pick='hash', learn_key=True):
        super(BinaryHashLinear, self).__init__()
        self.key_pick = key_pick
        self.n_in = n_in
        self.n_out = n_out
        w = nn.init.xavier_normal_(torch.empty(n_in, n_out))
        self.w = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(n_out))
        
        rand_01 = np.random.binomial(p=.5, n=1, size=(n_in, period)).astype(np.float32)
        o = torch.from_numpy(rand_01*2 - 1)
        self.o = nn.Parameter(o)
        if not learn_key:
            self.o.requires_grad = False

    def forward(self, x, key_id_list):
        '''
        corrected to include array of key_ids 
        (May be different for each x sample - row)
        key_id_list (list): must have len == x.shape[0]
        '''
        assert len(key_id_list) == x.shape[0]
        o = self.o[:, key_id_list].T
        # print(o.shape, x.shape)
        m = x*o
        r = torch.mm(m, self.w)
        return r

    

load contrastive backbone


In [25]:
images = torch.rand(size=(10,2048))

predict_task_baseline_peakedness(3, images, network, peakedness_m='max')

Y_hat torch.Size([10, 104])
Y_hat torch.Size([10, 72])
Y_hat torch.Size([10, 38])
values tensor([0.0141, 0.0198, 0.0419], device='cuda:0')


2

In [ ]:
images = torch.rand(size=(10,2048))
images = images.cuda()
task_embedding = network(images, 2, base_apply=False)


In [12]:
for tt in range(2):
    print(tt)

0
1
